In [1]:
import cv2
import mediapipe as mp
import numpy as np
import csv
import pandas as pd

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End


    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    #c[0]==>x    c[1]==>y
    angle = np.abs(radians*180.0/np.pi)

    if angle >180.0:
        angle = 360-angle

    return angle 



url = "http://192.168.1.57:4747/video"
cap = cv2.VideoCapture(url)

# cap = cv2.VideoCapture(0)
num_correct_moves=0
stage = None
in_range = False
prev_arm_angle = None

correct_moves=[]
angles=[]


## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:    
    with open('new.csv', mode='w', newline='') as csv_file:
        
        writer = csv.writer(csv_file)
        writer.writerow(['number of correct moves', 'angle'])

        while cap.isOpened():
            ret, frame = cap.read()

            # Recolor image to RGB
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False

            # Make detection
            results = pose.process(image)

            # Recolor back to BGR
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            # Extract landmarks
            try:

                landmarks = results.pose_landmarks.landmark

                shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
                elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
                wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

                angle = calculate_angle(shoulder, elbow, wrist)

                cv2.putText(image, str(int(angle)), 
                            tuple(np.multiply(elbow, [640, 480]).astype(int)), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                    )


                if 130 <= angle <= 154:
                    if in_range and prev_arm_angle is not None and prev_arm_angle <= 154:
                        pass
                    else:
                        in_range = True
                        
                        # Check if num_correct_moves is already present in correct_moves list
                        if (num_correct_moves not in correct_moves) and (num_correct_moves != 0) :
                            writer.writerow([num_correct_moves, angle])
                            correct_moves.append(num_correct_moves)
                        
                        num_correct_moves += 1
                        stage="right moves"
                        stage_color = (255, 255, 255)
                else:
                    if in_range:
                        writer.writerow(['Right range exceeded'])
                        stage="Right range exceeded"
                        stage_color = (0, 0, 255)
                        in_range = False

                prev_arm_angle = angle
  
            except:
                pass
            
                            
#             df = pd.read_csv('new.csv')
#             df.to_excel('output_file.xlsx', index=False)


            # Setup status box
            cv2.rectangle(image, (0,0), (380,73), (245,117,16), -1)


            # Stage and reps data
            if stage == "Right range exceeded":
                cv2.putText(image, stage, 
                            (15,50), 
                            cv2.FONT_HERSHEY_DUPLEX, 1, stage_color, 2, cv2.LINE_AA)
            else:
                cv2.putText(image, 'REPS', (0,12), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
                cv2.putText(image, str(num_correct_moves), 
                            (2,60), 
                            cv2.FONT_HERSHEY_SCRIPT_COMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
                cv2.putText(image, 'STAGE', (100,12), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
                cv2.putText(image, stage, 
                            (50,60), 
                            cv2.FONT_HERSHEY_SCRIPT_COMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)




            # Render detections
#             mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
#                                     mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
#                                     mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
#                                     )               

            cv2.imshow('Final', image)
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break

        cap.release()
        cv2.destroyAllWindows()